# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [18]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [19]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [21]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [22]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [23]:
messages = [
    {"role": "system", "content": "use tone as a middle school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. It's an important city where the President lives and where you'll find a lot of the country's important government buildings, like the White House and the Capitol. If you ever get the chance to visit, there's a lot to learn and see!


Add assistant messages to teach LLM what `##` is.

In [24]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [25]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [26]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [27]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [28]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])
    print (tweet_data)

["Couldn't you quote your idol Mao, or would you fumble the election? https://t.co/W6fuRj0b2k https://t.co/UBzwK0mMUq"]
["Couldn't you quote your idol Mao, or would you fumble the election? https://t.co/W6fuRj0b2k https://t.co/UBzwK0mMUq", 'The media and the left spend all year claiming crime is down. They “fact check” Trump at the debate on it. Weeks before the election, the FBI quietly “updates” the data to show that crime is in fact up. The leftist response is to say “relax, rightoid, the FBI does this REGULARLY”']
["Couldn't you quote your idol Mao, or would you fumble the election? https://t.co/W6fuRj0b2k https://t.co/UBzwK0mMUq", 'The media and the left spend all year claiming crime is down. They “fact check” Trump at the debate on it. Weeks before the election, the FBI quietly “updates” the data to show that crime is in fact up. The leftist response is to say “relax, rightoid, the FBI does this REGULARLY”', 'RT @NickJFuentes: Trump 2024:\n• Mass Immigration (legally)\n• Trans Ki

In [29]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  200


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [30]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets cover a wide range of topics related to elections, primarily focusing on the U.S. political landscape. Key themes include allegations of election interference, particularly involving the FBI and media, and concerns about voter fraud and election integrity. There are discussions about Donald Trump's potential 2024 candidacy, with some predicting a strong performance and others expressing skepticism about polling accuracy. The tweets also touch on the role of media in shaping election narratives, the influence of dark money, and the importance of voter turnout. Additionally, there are mentions of international election-related issues, such as India's political dynamics and potential geopolitical actions by Israel. Overall, the tweets reflect a mix of political opinions, predictions, and concerns about the upcoming elections.


In [31]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss election-related topics, including Trump, crime statistics, media bias, voter registration issues, and potential election interference.


In [32]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets discuss AI in the context of election security and misinformation. Concerns are raised about AI's role in spreading false information and influencing public opinion. Some tweets suggest AI could be used to manipulate election outcomes, while others highlight the need for transparency and security in AI applications related to elections.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [33]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [34]:
for tweet in tweet_data:
    flag_help(tweet)

===
harassment
The media and the left spend all year claiming crime is down. They “fact check” Trump at the debate on it. Weeks before the election, the FBI quietly “updates” the data to show that crime is in fact up. The leftist response is to say “relax, rightoid, the FBI does this REGULARLY”
===
harassment
@DC_Draino Kamala let in a bunch of terrorist! DHS caught one planning an attack on election day! He was in oklahoma of all places! They are everywhere!
===
harassment
RT @maddenifico: Some of you self-righteous motherfuckers on the left are once again overthinking this. It's as if you faux-prog frauds lea…
===
harassment
@LauraLoomer Their time will be coming to an end soon once Trump wins this election. They will be running and hiding. No more protection from this treasonous administration.
===
harassment
RT @Starboy2079: Why BJP doesn't understand a simple fact that as number of Bangladeshi Muslims and Rohingyas will keep increasing, their p…
===
violence
RT @ecomarxi: Biden/Ha

### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [35]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

你不能引用你的偶像毛泽东的话吗，还是你会搞砸选举？https://t.co/W6fuRj0b2k https://t.co/UBzwK0mMUq
媒体和左派整年都在声称犯罪率下降。他们在辩论中对特朗普的说法进行“事实核查”。在选举前几周，FBI悄悄“更新”数据，显示犯罪率实际上在上升。左派的回应是说“放轻松，右派，FBI经常这样做”
转发 @NickJFuentes: 特朗普2024： • 大规模移民（合法） • 跨性别儿童（需父母同意） • 不发动新战争（伊朗除外） • 免费…
如果唐纳德·特朗普赢得选举，有两种可能的情况：

1. 加密货币市场因我们终于有了一位支持加密货币的总统而上涨。

2. 市场出现小幅下跌，“买传闻，卖事实”。

从长远来看，无论谁是总统都完全无关紧要，因为加密货币是…… https://t.co/8hqZ5YNQmQ
@joshtheobald3 我们必须这样做，妻子在选举日工作，她下班太晚，来不及赶上，我们将在21日投票。
在他们的选举报道节目中，India Today将埃克纳特·辛德描绘为马拉塔士兵，而乌达夫·萨克雷则被描绘为莫卧儿士兵。

也许这是一个弗洛伊德式的失误，或者可能是故意的，但似乎India Today这次准确地把握了基层的情绪。 https://t.co/LiXvIYzSYI
🇺🇸🚨 我们正在前往默瑟县选举委员会的路上...

显然，那里的一个无家可归居民被拒绝注册为选民，这在宾夕法尼亚州是非法的。

今晚我们将和平地参加那里的选举委员会听证会，以表达我们的不满。
@DC_Draino 卡玛拉放进了一群恐怖分子！国土安全部抓住了一个计划在选举日发动袭击的人！他居然在俄克拉荷马！他们无处不在！
@ssecijak 我们必须以压倒性的数量出现，以毫无疑问地表明谁赢得了这次选举。
FBI发布虚假的犯罪统计数据，然后各大网络使用这些虚假数据在辩论中对特朗普进行“事实核查”，而实际上他在说真话？这就是FBI和新闻网络的实际选举干预。这是叛国。这是反美。这是战争。
@christianboutin @scrowder 他们正在推动这些虚假的民意调查，显示特朗普大幅领先，这样当卡玛拉在十一月以压倒性优势获胜时，他们就可以声称选举舞弊。
RT @whstancil: 生活在摇摆州的人们：我们正被右翼虚假信息淹没。所有大型国家媒体都拒

KeyboardInterrupt: 

In [36]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Franklin Saint """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

Couldn't you drop some wisdom from your hero Mao, or are you too shook to handle the election heat? https://t.co/W6fuRj0b2k https://t.co/UBzwK0mMUq
Man, they always try to play us like we ain't paying attention. The media and the left keep pushing that narrative, saying crime's down, trying to fact-check Trump like they got all the answers. But then, right before the election, the FBI drops some new numbers showing crime's actually up. And what do they say? "Chill out, this is just how it goes." Nah, we see the game.
RT @NickJFuentes: Trump 2024: 
• Letting folks in, but doing it by the book 
• Kids making choices, but only if the folks say so 
• Keeping the peace, unless it’s with Iran 
• Free Spe…
If Trump takes the White House, we got two plays on the table:

1. Crypto might just skyrocket 'cause we got a President who’s all about that digital hustle.

2. Or, we see a little dip, 'cause folks cash out on the hype.

But let’s be real, in the long run, it don’t matter who’s sitting in

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [37]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "negative",
  "emotion": "sarcasm",
  "mentioned": [],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "frustration",
  "mentioned": ["Trump", "FBI"],
  "support": "Republican"
}
{
  "sentiment": "negative",
  "emotion": "frustration",
  "mentioned": ["NickJFuentes", "Trump"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "speculative",
  "mentioned": ["Donald Trump"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "determination",
  "mentioned": ["@joshtheobald3"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "curiosity",
  "mentioned": [
    "Eknath Shinde",
    "Uddhav Thackeray",
    "India Today"
  ],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "concern",
  "mentioned": [],
  "support": "Democratic"
}
{
  "sentiment": "negative",
  "emotion": "fear",
  "mentioned": ["Kamala"],
  "support": "Republican"
}
{
  "sentiment": "neutral",
  "emotion": "determination

In [38]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "topics": [
    "Election Interference",
    "Trump's 2024 Campaign",
    "Voting and Voter Registration Issues",
    "Media and Misinformation",
    "Crime Statistics and FBI Data",
    "Crypto Market and Elections",
    "Political Polling and Predictions",
    "Election Security and Integrity",
    "Political Campaigns and Strategies",
    "International Relations and Elections"
  ]
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [39]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 200/200 [05:52<00:00,  1.76s/it]


In [40]:
print(analysis_result)

['{\n  "sentiment": "negative",\n  "emotion": "sarcasm",\n  "mentioned": [],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "frustration",\n  "mentioned": ["Trump", "FBI"],\n  "support": "Republican"\n}', '{\n  "sentiment": "negative",\n  "emotion": "criticism",\n  "mentioned": ["NickJFuentes", "Trump"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "speculative",\n  "mentioned": ["Donald Trump"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "determination",\n  "mentioned": ["@joshtheobald3"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "curiosity",\n  "mentioned": ["Eknath Shinde", "Uddhav Thackeray"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "concern",\n  "mentioned": [],\n  "support": "Democratic"\n}', '{\n  "sentiment": "negative",\n  "emotion": "fear",\n  "mentioned": ["Kamala"],\n  "support": "Republican"\n}', '{\n  "sentiment": "neu

In [41]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

{
  "Democratic count": 25,
  "Republican count": 54,
  "people name": {
    "Democratic": {
      "common sentiments": ["negative", "neutral", "positive"],
      "common emotions": ["frustration", "concern", "informative", "admiration", "excitement", "satisfaction", "hopeful", "supportive"]
    },
    "Republican": {
      "common sentiments": ["negative", "neutral", "positive"],
      "common emotions": ["frustration", "anger", "informative", "anticipation", "outrage", "skepticism", "concern", "excitement"]
    }
  }
}


## Create a chatbot

In [42]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")

## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
